In [1]:
import os
import sys
import numpy as np
import matplotlib
import pandas as pd
import geopandas as gpd
import json
import pickle
import xgboost
import torch
import pickle

#import arcpy

In [2]:
path = '../../data'

In [3]:
def preprocess_eval_data(files):
    pass

In [ ]:
def preprocess_train_data(files):
    """
    
    description
    
    param files: 
    """
    
    # set up env
    arcpy.env.workspace = '../../data'
    arcpy.env.overwriteOutput = True
    save_path = '../../scratch/'
    
    # gather the shape files
    files = np.array(files)
    obs = files[0]
    rnd = files[1]
    
    # gather the raster files
    rasters = files[2:]
    rasters = list(rasters[rasters != '#'])

    # sample rasters from observed points
    obs_table = save_path + 'observed_presence_sampled.shp'
    obs_points = arcpy.sa.Sample(rasters, obs, obs_table)
    obs_count = int(arcpy.management.GetCount(obs_table)[0])
    obs_label = np.array([1]*obs_count)
    
    # sample rasters from absence points
    rnd_table = save_path + 'random_absence_sampled.shp'
    rnd_points = arcpy.sa.Sample(rasters, rnd, rnd_table)
    rnd_count = int(arcpy.management.GetCount(rnd_table)[0])
    rnd_label = np.array([0]*rnd_count)

    return obs_table, obs_label, rnd_table, rnd_label


preprocess_train_data(['V:\\ENV859_Final_Project_al512\\data\\brf_obs.shp', 'V:\\ENV859_Final_Project_al512\\data\\rand_obs.shp', 'V:\\ENV859_Final_Project_al512\\data\\bathy', 'V:\\ENV859_Final_Project_al512\\data\\habras10_1', 'V:\\ENV859_Final_Project_al512\\data\\habras10_2', 'V:\\ENV859_Final_Project_al512\\data\\habras10_3', 'V:\\ENV859_Final_Project_al512\\data\\habras10_4', 'V:\\ENV859_Final_Project_al512\\data\\habras10_5', 'V:\\ENV859_Final_Project_al512\\data\\habras10_6', 'V:\\ENV859_Final_Project_al512\\data\\habras10_7', 'V:\\ENV859_Final_Project_al512\\data\\habras10_8', 'V:\\ENV859_Final_Project_al512\\data\\dist_kelp', 'V:\\ENV859_Final_Project_al512\\data\\dist_100m', 'V:\\ENV859_Final_Project_al512\\data\\botc10_8ws', '#', '#'])

In [ ]:
"../../scratch/random_absence_sampled.dbf"

In [53]:
import xgboost as xgb
import pickle
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from simpledbf import Dbf5
import pandas as pd
#import preprocess_data

In [51]:
def build_model(files):
    """
    Description
    
    param files: 
    """
    # preprocess data
    obs_path, obs_label, rnd_path, rnd_label = files
    
    # read training file
    obs_dbf = Dbf5(obs_path)
    rnd_dbf = Dbf5(rnd_path)
    obs_df = obs_dbf.to_dataframe()
    rnd_df = rnd_dbf.to_dataframe()

    # prepare data for training
    obs_df = obs_df.drop(columns = ['brf_obs', 'X', 'Y'])
    rnd_df = rnd_df.drop(columns = ['rand_obs', 'X', 'Y'])
    obs_df['label'] = obs_label
    rnd_df['label'] = rnd_label
    data = pd.concat([obs_df, rnd_df])
    X = data.drop(columns = ['label'])
    Y = data['label']
    
    # build xgboost model
    clf = XGBClassifier(objective= 'binary:logistic')
    parameters = {
        'max_depth': range (2, 10),
        'n_estimators': range(60, 220, 40),
        'learning_rate': [0.1, 0.01, 0.05]
    }
    grid_search = GridSearchCV(
        estimator = clf,
        param_grid = parameters,
        scoring = 'roc_auc',
        n_jobs = 5,
        cv = 5,
        verbose = True)
    grid_search.fit(X, Y)
    
    model_path = "../../scratch/xgb_model.pkl"

In [52]:
files = ["../../scratch/observed_presence_sampled.dbf", 
                                                [1]*193, 
                                                "../../scratch/random_absence_sampled.dbf", 
                                                [0]*193]
build_model(files)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=No

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
obs_df['label'] = [1]*193
rnd_df['label'] = [0]*193

In [22]:
rnd_df = rnd_df.drop(columns = ['rand_obs', 'X', 'Y'])

In [23]:
pd.concat([obs_df, rnd_df])

,bathy,habras10_1,habras10_2,habras10_3,habras10_4,habras10_5,habras10_6,habras10_7,habras10_8,dist_kelp,dist_100m,botc10_8ws,label
0,-36.9445,0,1,0,0,0,0,0,0,470.106,590.762,35,1
1,-36.3049,0,1,0,0,0,0,0,0,475.395,593.633,33,1
2,-36.3049,0,1,0,0,0,0,0,0,475.395,593.633,33,1
3,-36.3049,0,1,0,0,0,0,0,0,475.395,593.633,33,1
4,-35.9506,0,1,0,0,0,0,0,0,477.598,605.392,32,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,-39.7903,0,0,0,1,0,0,0,0,590.339,672.309,32,0
189,-36.4175,1,0,0,0,0,0,0,0,371.214,959.010,35,0
190,-31.0446,1,0,0,0,0,0,0,0,383.275,586.941,38,0
191,-27.6483,1,0,0,0,0,0,0,0,543.323,1015.330,33,0


In [ ]:
if __name__ == "__main__":
    file_inputs = json.loads(sys.argv[0])